In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
pip install pingouin

In [ ]:
import numpy as np

EPSILON = 1e-10


def rae(actual: np.ndarray, predicted: np.ndarray):
    """ Relative Absolute Error (aka Approximation Error) """
    return np.sum(np.abs(actual - predicted)) / (np.sum(np.abs(actual - np.mean(actual))) + EPSILON)

In [ ]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

df = pd.read_csv('anon_jobs_final.csv')

df['QueueID'] = df['QueueID'].str[1:].astype(int)
df['ExecutableID'] = df['ExecutableID'].str[1:].astype(int)
df['GroupID'] = df['GroupID'].str[1:].astype(int)


non_historical_data=df[["SubmitTime","UsedMemory","ReqNProcs","ReqTime","ReqMemory","UserID","GroupID","QueueID","OrigSiteID"]]
target=df[["RunTime"]]

le = LabelEncoder()
le.fit(non_historical_data['UserID'].astype(str))
non_historical_data['UserID'] = le.transform(non_historical_data['UserID'].astype(str))

le.fit(non_historical_data['OrigSiteID'].astype(str))
non_historical_data['OrigSiteID'] = le.transform(non_historical_data['OrigSiteID'].astype(str))


X_train, X_test, y_train, y_test = train_test_split(non_historical_data, target, test_size=0.33, random_state=42)

In [ ]:
y_train=y_train.values.reshape(-1)

In [ ]:
y_test=y_test.values.reshape(-1)

In [ ]:
from sklearn.preprocessing import StandardScaler
sc_X = StandardScaler()
X_trainscaled=sc_X.fit_transform(X_train)
X_testscaled=sc_X.transform(X_test)

In [ ]:
from sklearn.neural_network import MLPRegressor
reg = MLPRegressor(hidden_layer_sizes=(64,64,64),activation="relu" ,random_state=1, max_iter=2000).fit(X_trainscaled, y_train)

In [ ]:
y_pred=reg.predict(X_testscaled)

In [ ]:
import pingouin as pg
pg.corr(y_test,y_pred)

,n,r,CI95%,r2,adj_r2,p-val,BF10,power
pearson,114712,0.807306,"[0.81, 0.81]",0.651743,0.651737,0.0,inf,1.0


In [ ]:
rae(y_test,y_pred)

0.32795940628861614

MLP (16,16,16)

In [ ]:
from sklearn.neural_network import MLPRegressor
reg = MLPRegressor(hidden_layer_sizes=(16,16,16),activation="relu" ,random_state=1, max_iter=2000).fit(X_trainscaled, y_train)

In [ ]:
y_pred=reg.predict(X_testscaled)

In [ ]:
import pingouin as pg
pg.corr(y_test,y_pred)

,n,r,CI95%,r2,adj_r2,p-val,BF10,power
pearson,114712,0.773203,"[0.77, 0.78]",0.597843,0.597836,0.0,inf,1.0


In [ ]:
rae(y_test,y_pred)

0.36766779033854613

MLP (32,32,32)

In [ ]:
from sklearn.neural_network import MLPRegressor
reg = MLPRegressor(hidden_layer_sizes=(32,32,32),activation="relu" ,batch_size=32,random_state=1, max_iter=2000).fit(X_trainscaled, y_train)

In [ ]:
y_pred=reg.predict(X_testscaled)

In [ ]:
import pingouin as pg
pg.corr(y_test,y_pred)

,n,r,CI95%,r2,adj_r2,p-val,BF10,power
pearson,114712,0.811859,"[0.81, 0.81]",0.659115,0.659109,0.0,inf,1.0


In [ ]:
rae(y_test,y_pred)

0.34170677025743174

In [ ]:
check_parameters = {
    'hidden_layer_sizes': [(50,50), (100,50)],
    'activation': ['tanh', 'relu'],
    'solver': ['sgd', 'adam'],
    'alpha': [0.0001, 0.05],
    'learning_rate': ['constant','adaptive'],
}

In [ ]:
from sklearn.model_selection import GridSearchCV
gridsearchcv=GridSearchCV(MLPRegressor,check_parameters,n_jobs=-1,cv=3)

XGBOOST MAX DEPTH=4

In [ ]:
import xgboost as xg
xgb_r = xg.XGBRegressor(max_depth=4)
xgb_r.fit(X_train,y_train)

[16:45:18] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0,
             importance_type='gain', learning_rate=0.1, max_delta_step=0,
             max_depth=4, min_child_weight=1, missing=None, n_estimators=100,
             n_jobs=1, nthread=None, objective='reg:linear', random_state=0,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
             silent=None, subsample=1, verbosity=1)

In [ ]:
pred=xgb_r.predict(X_test)

In [ ]:
import pingouin as pg
pg.corr(y_test,pred)

,n,r,CI95%,r2,adj_r2,p-val,BF10,power
pearson,114712,0.801032,"[0.8, 0.8]",0.641652,0.641646,0.0,inf,1.0


In [ ]:
rae(y_test,pred)

0.37017480164864003

XGBOOST MAX DEPT=3

In [ ]:
import xgboost as xg
xgb_r = xg.XGBRegressor()
xgb_r.fit(X_train,y_train)

[14:07:58] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0,
             importance_type='gain', learning_rate=0.1, max_delta_step=0,
             max_depth=3, min_child_weight=1, missing=None, n_estimators=100,
             n_jobs=1, nthread=None, objective='reg:linear', random_state=0,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
             silent=None, subsample=1, verbosity=1)

In [ ]:
pred=xgb_r.predict(X_test)

In [ ]:
import pingouin as pg
pg.corr(y_test,pred)

,n,r,CI95%,r2,adj_r2,p-val,BF10,power
pearson,114712,0.745592,"[0.74, 0.75]",0.555907,0.555899,0.0,inf,1.0


In [ ]:
rae(y_test,pred)

0.4307013483452035

XGBOOST MAX DEPTH=8

In [ ]:
import xgboost as xg
xgb_r = xg.XGBRegressor(max_depth=8)
xgb_r.fit(X_train,y_train)

[16:42:07] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0,
             importance_type='gain', learning_rate=0.1, max_delta_step=0,
             max_depth=8, min_child_weight=1, missing=None, n_estimators=100,
             n_jobs=1, nthread=None, objective='reg:linear', random_state=0,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
             silent=None, subsample=1, verbosity=1)

In [ ]:
pred=xgb_r.predict(X_test)

In [ ]:
import pingouin as pg
pg.corr(y_test,pred)

,n,r,CI95%,r2,adj_r2,p-val,BF10,power
pearson,114712,0.886513,"[0.89, 0.89]",0.785905,0.785901,0.0,inf,1.0


In [ ]:
rae(y_test,pred)

0.22837458981100314

XGBOOST MAX DEPTH=12

In [ ]:
import xgboost as xg
xgb_r = xg.XGBRegressor(max_depth=12)
xgb_r.fit(X_train,y_train)

[14:01:31] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0,
             importance_type='gain', learning_rate=0.1, max_delta_step=0,
             max_depth=12, min_child_weight=1, missing=None, n_estimators=100,
             n_jobs=1, nthread=None, objective='reg:linear', random_state=0,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
             silent=None, subsample=1, verbosity=1)

In [ ]:
pred=xgb_r.predict(X_test)

In [ ]:
import pingouin as pg
pg.corr(y_test,pred)

,n,r,CI95%,r2,adj_r2,p-val,BF10,power
pearson,114712,0.903997,"[0.9, 0.91]",0.817211,0.817208,0.0,inf,1.0


In [ ]:
rae(y_test,pred)

0.1735207596966376

XGBOOST MAX DEPTH=15

In [ ]:
import xgboost as xg
xgb_r = xg.XGBRegressor(max_depth=15)
xgb_r.fit(X_train,y_train)

[14:07:27] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0,
             importance_type='gain', learning_rate=0.1, max_delta_step=0,
             max_depth=15, min_child_weight=1, missing=None, n_estimators=100,
             n_jobs=1, nthread=None, objective='reg:linear', random_state=0,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
             silent=None, subsample=1, verbosity=1)

In [ ]:
pred=xgb_r.predict(X_test)

In [ ]:
import pingouin as pg
pg.corr(y_test,pred)

,n,r,CI95%,r2,adj_r2,p-val,BF10,power
pearson,114712,0.904668,"[0.9, 0.91]",0.818425,0.818422,0.0,inf,1.0


In [ ]:
rae(y_test,pred)

0.15924360356448894

XGBOOST max depth = 18

In [ ]:
import xgboost as xg
xgb_r = xg.XGBRegressor(max_depth=18)
xgb_r.fit(X_train,y_train)

[09:34:07] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0,
             importance_type='gain', learning_rate=0.1, max_delta_step=0,
             max_depth=18, min_child_weight=1, missing=None, n_estimators=100,
             n_jobs=1, nthread=None, objective='reg:linear', random_state=0,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
             silent=None, subsample=1, verbosity=1)

In [ ]:
pred=xgb_r.predict(X_test)

In [ ]:
import pingouin as pg
pg.corr(y_test,pred)

,n,r,CI95%,r2,adj_r2,p-val,BF10,power
pearson,114712,0.901677,"[0.9, 0.9]",0.813022,0.813019,0.0,inf,1.0


In [ ]:
rae(y_test,pred)

0.1514286693456375

KNN with n=3

In [ ]:
from sklearn.neighbors import KNeighborsRegressor
knn = KNeighborsRegressor(n_neighbors=2)
knn.fit(X_train, y_train)

KNeighborsRegressor(algorithm='auto', leaf_size=30, metric='minkowski',
                    metric_params=None, n_jobs=None, n_neighbors=2, p=2,
                    weights='uniform')

In [ ]:
y_pred=knn.predict(X_test)

In [ ]:
import pingouin as pg
pg.corr(y_test,y_pred)

,n,r,CI95%,r2,adj_r2,p-val,BF10,power
pearson,114712,0.833805,"[0.83, 0.84]",0.695231,0.695226,0.0,inf,1.0


In [ ]:
rae(y_test,y_pred)

0.20423348403510744

In [ ]:
from sklearn.neighbors import KNeighborsRegressor
knn = KNeighborsRegressor(n_neighbors=3)
knn.fit(X_train, y_train)

KNeighborsRegressor(algorithm='auto', leaf_size=30, metric='minkowski',
                    metric_params=None, n_jobs=None, n_neighbors=3, p=2,
                    weights='uniform')

In [ ]:
y_pred=knn.predict(X_test)

In [ ]:
import pingouin as pg
pg.corr(y_test,y_pred)

,n,r,CI95%,r2,adj_r2,p-val,BF10,power
pearson,114712,0.843082,"[0.84, 0.84]",0.710787,0.710782,0.0,inf,1.0


In [ ]:
rae(y_test,y_pred)

0.20649698443231113

KNN with n=5

In [ ]:
from sklearn.neighbors import KNeighborsRegressor
knn = KNeighborsRegressor(n_neighbors=5)
knn.fit(X_train, y_train)
y_pred=knn.predict(X_test)


In [ ]:
import pingouin as pg
pg.corr(y_test,y_pred)

,n,r,CI95%,r2,adj_r2,p-val,BF10,power
pearson,114712,0.853632,"[0.85, 0.86]",0.728687,0.728682,0.0,inf,1.0


In [ ]:
rae(y_test,y_pred)

0.21235532364007487

KNN with n=7

In [ ]:
from sklearn.neighbors import KNeighborsRegressor
knn = KNeighborsRegressor(n_neighbors=7)
knn.fit(X_train, y_train)
y_pred=knn.predict(X_test)

In [ ]:
import pingouin as pg
pg.corr(y_test,y_pred)

,n,r,CI95%,r2,adj_r2,p-val,BF10,power
pearson,114712,0.854252,"[0.85, 0.86]",0.729746,0.729742,0.0,inf,1.0


In [ ]:
rae(y_test,y_pred)

0.2197510663654613

KNN with n=9

In [ ]:
from sklearn.neighbors import KNeighborsRegressor
knn = KNeighborsRegressor(n_neighbors=9)
knn.fit(X_train, y_train)
y_pred=knn.predict(X_test)

In [ ]:
import pingouin as pg
pg.corr(y_test,y_pred)

,n,r,CI95%,r2,adj_r2,p-val,BF10,power
pearson,114712,0.85075,"[0.85, 0.85]",0.723775,0.723771,0.0,inf,1.0


In [ ]:
rae(y_test,y_pred)

0.22747493177836997

RANDOM FOREST N_ESTIMATORS=100

In [ ]:
from sklearn.ensemble import RandomForestRegressor
model = RandomForestRegressor()
model.fit(X_train, y_train)

RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mse',
                      max_depth=None, max_features='auto', max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      n_estimators=100, n_jobs=None, oob_score=False,
                      random_state=None, verbose=0, warm_start=False)

In [ ]:
y_pred=model.predict(X_test)

In [ ]:
import pingouin as pg
pg.corr(y_test,y_pred)

,n,r,CI95%,r2,adj_r2,p-val,BF10,power
pearson,114712,0.903091,"[0.9, 0.9]",0.815574,0.815571,0.0,inf,1.0


In [ ]:
rae(y_test,y_pred)

0.15101921946379288

RANDOM FOREST N_ESTIMATORS=500



In [ ]:
from sklearn.ensemble import RandomForestRegressor
model = RandomForestRegressor(n_estimators=500)
model.fit(X_train, y_train)

RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mse',
                      max_depth=None, max_features='auto', max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      n_estimators=500, n_jobs=None, oob_score=False,
                      random_state=None, verbose=0, warm_start=False)

In [ ]:
y_pred=model.predict(X_test)

In [ ]:
import pingouin as pg
pg.corr(y_test,y_pred)

,n,r,CI95%,r2,adj_r2,p-val,BF10,power
pearson,114712,0.903077,"[0.9, 0.9]",0.815549,0.815545,0.0,inf,1.0


In [ ]:
rae(y_test,y_pred)

0.1525730955927816

RANDOM FOREST N_ESTIMATORS=200

In [ ]:
from sklearn.ensemble import RandomForestRegressor
model = RandomForestRegressor(n_estimators=50)
model.fit(X_train, y_train)

RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mse',
                      max_depth=None, max_features='auto', max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      n_estimators=50, n_jobs=None, oob_score=False,
                      random_state=None, verbose=0, warm_start=False)

In [ ]:
y_pred=model.predict(X_test)

In [ ]:
import pingouin as pg
pg.corr(y_test,y_pred)

,n,r,CI95%,r2,adj_r2,p-val,BF10,power
pearson,114712,0.901072,"[0.9, 0.9]",0.81193,0.811927,0.0,inf,1.0


In [ ]:
rae(y_test,y_pred)

0.15148583003044735

DECISION TREE REGRESSOR

In [ ]:
from sklearn.tree import DecisionTreeRegressor
model = DecisionTreeRegressor()
model.fit(X_train, y_train)

DecisionTreeRegressor(ccp_alpha=0.0, criterion='mse', max_depth=None,
                      max_features=None, max_leaf_nodes=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=1, min_samples_split=2,
                      min_weight_fraction_leaf=0.0, presort='deprecated',
                      random_state=None, splitter='best')

In [ ]:
y_pred=model.predict(X_test)

In [ ]:
import pingouin as pg
pg.corr(y_test,y_pred)

,n,r,CI95%,r2,adj_r2,p-val,BF10,power
pearson,114712,0.858708,"[0.86, 0.86]",0.73738,0.737375,0.0,inf,1.0


In [ ]:
rae(y_test,y_pred)

0.1710921348768146